In [ ]:
import compas.geometry as cg
from arch import construct_arch
from problem.variables import Problem
from problem.helpers import print_matrix
import numpy as np
import cvxpy as cp
from compas_viewer import Viewer
from compas.colors import Color
import copy
from compas_viewer.scene.tagobject import Tag
from lintles import construct_lintles


In [9]:
(8) //4

2

In [ ]:
n = 10
height = 3000
span = 10000
thickness = 1000
Arch_Object = construct_arch(height, span, n, thickness)
Arch_Object_0 = copy.deepcopy(Arch_Object)
The_Problem = Problem.from_polygons(Arch_Object)
The_Problem.A_eq

In [ ]:
c = The_Problem.C # to avoid singularities
#c[6] = 1.0  # small force to avoid singularity
# print("The Problem C vector:\n", c)
u = The_Problem.U
intfn = The_Problem.intfn
A_ub = The_Problem.A_ub
A_eq = The_Problem.A_eq

b_ub = np.zeros(A_ub.shape[0])
b_eq = np.zeros(A_eq.shape[0])

# print(f"number of interfaces: {intfn}")
def set_BCs(b, Idx, vector):
    """Sets the boundary conditions in the RHS vector b"""
    b[Idx*2    ] = vector[0]
    b[Idx*2 + 1] = vector[1]
    return b

def set_force_BCs(c, Idx, vector):
    """Set force boundary conditions in the global force vector C"""
    c[Idx*3    ] = vector[0]
    c[Idx*3 + 1] = vector[1]
    c[Idx*3 + 2] = vector[2]
    return c
g = -9.81
for i in range(n):
    set_force_BCs(c, i, (0, g, 0.0))

# index_BC_x = [0, n-1]
# for i in index_BC_x:
#     set_force_BCs(c, i, (-1.0, 0.0, -5))

#set_force_BCs(c, 2, (0.0, 0.0, 1.0))
#set_force_BCs(c, n-2, (0.0, 0.0, 1.0))

# set_BCs(b_ub, 0, (0.04, 000));
set_BCs(b_ub, 0, (-500, -500));
# set_BCs(b_eq, n-2, (0.05, 0.05));
# set_BCs(b_eq, n-2, (0.05, 0.05));
# set_BCs(b_ub, n-2, (0.1, 0.0));

# set_BCs(b_eq, 0, (-0.001, -0.001))
# set_BCs(b_ub, intfn-3, (0.01, 0))
# set_BCs(b_eq, intfn-3, (0.0, 0))


In [ ]:
U = cp.Variable(3*n)
# BC on displacements and rotations of block i
# i * 3 + 0 -> dx
# i * 3 + 1 -> dy
# i * 3 + 2 -> rz

constraints = [
    A_ub @ U >= b_ub,
    A_eq @ U == b_eq,
]

objective = cp.Maximize(c @ U)  # minimize −c·U  (equiv. maximize c·U)
prob = cp.Problem(objective, constraints)

prob.solve(solver=cp.MOSEK, verbose=False)  # or CBC / ECOS

print("Feasible:", prob.status)
U_star = U.value
# print(U_star)

A_eq_U = A_eq @ U_star
A_ub_U = A_ub @ U_star
# print_matrix(A_eq_U)
slip = A_eq @ U_star - b_eq
print(np.max(np.abs(slip)))

gap = A_ub @ U_star - b_ub
print(np.min(gap))
print_matrix(A_eq_U)


In [ ]:
U_new = U_star.reshape((n, 3))

def deform_polygon_linear(poly, Ux, Uz, th):
    c = poly.centroid()
    cx, cz = c[0], c[2]

    new_pts = []
    for p in range(4):  # or poly.vertices, depending on compas version
        x, z = poly.vertex[p]["x"], poly.vertex[p]["z"]
        x_new = x + Ux - th*(z - cz)
        z_new = z + Uz + th*(x - cx)
        new_pts.append(cg.Point(x_new, 0.0, z_new))

    return cg.Polygon(new_pts)

# build deformed polygons for viewing
lintl_def = []
for i, poly in enumerate(Arch_Object_0):          # use original undeformed copy
    dx, dz, th = U_new[i]                   # your (Ux, Uz, theta)
    lintl_def.append(deform_polygon_linear(poly, dx, dz, th))


scale = 0.001  # scale factor for visualization
viewer = Viewer()
for idx, polygon in enumerate(lintl_def):
    polygon.scale(scale)
    viewer.scene.add(polygon,facecolor=Color(0.0, 0.0, 1.0), show_points=True, pointcolor=Color(1.0, 0.0, 1.0), linecolor=Color(1.0, 0.5, 1.0), linewidth=1, opacity=0.5)
    # dx, dy, rz = U_new[i]
    # string = f"dx: {dx:.3f} \n dy: {dy:.3f} \n rz: {rz:.3f}"
    # # string = idx
    # t1 = Tag(text=idx, position=polygon.centroid(), color=Color(1.0, 0.0, 0.0))
    # viewer.scene.add(t1)

viewer.show()


In [ ]:
# U_new = U_star.reshape((n, 3))
# # print_matrix(A_ub)
# print_matrix(U_new)
# for i,polygon in enumerate(Arch_Object):
#     dx, dy, rz = U_new[i]
#     polygon.translate(cg.Vector(dx, 0,dy))
#     polygon.rotate(rz, axis=cg.Vector(0,1, 0), point=polygon.centroid())



In [ ]:
# scale = 0.001  # scale factor for visualization
# viewer = Viewer()
# viewer.scene.clear()
# for idx, polygon in enumerate(Arch_Object):
#     polygon.scale(scale)
#     viewer.scene.add(polygon,facecolor=Color(0.0, 0.0, 1.0), show_points=True, pointcolor=Color(1.0, 0.0, 1.0), linecolor=Color(1.0, 0.5, 1.0), linewidth=1, opacity=0.5)
    
#     t1 = Tag(text=str(idx), position=polygon.centroid(), color=Color(1.0, 0.0, 0.0))
#     viewer.scene.add(t1)

#     # c_old = Arch_Object_0[idx].centroid()
#     # c_new = polygon.centroid()
#     # x_o, y_o, r_o = c_old[0], c_old[1], c_old[2]
#     # x_n, y_n, r_n = c_new[0], c_new[1], c_new[2]
#     # translation = cg.Vector(x_n - x_o, y_n - y_o, r_n - r_o)

#     # viewer.scene.add(translation, anchor=c_old, color=Color(0.0, 1.0, 0.50), linewidth=2)

# viewer.show()